In [105]:
import requests
import json
import pandas as pd
import numpy as np
from functools import reduce
import datetime
import wbdata
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook" # use "pio.renderers" to see the default renderer

## Objective
- Build table with yearly debt for a selection of countries. 
- Metric is called "Total External debt stocks" here: https://datatopics.worldbank.org/debt/ids/countryanalytical/ARG
- Data source: https://worldbank.github.io/debt-data/api-guide/ids-api-guide-python-1.html
- Source name: "International Debt Statistics", id:6
- Indicator name: "External debt stocks, total (DOD, current US$)", code: "DT.DOD.DECT.CD"
- Country selection: "Low & middle income" (LMY)
- World Bank Data API docs: https://wbdata.readthedocs.io/en/latest/wbdata_library.html

### Check data coordinates

In [6]:
# Make sure source exists
wbdata.get_source(source_id=6)

  id  name
----  -----------------------------
   6  International Debt Statistics

In [21]:
# Make sure indicator exists in source
ind = wbdata.get_indicator(source=6)
for i in ind:
    if i["id"] == "DT.DOD.DECT.CD":
        print("Found: "+ (str(i)))

Found: {'id': 'DT.DOD.DECT.CD', 'name': 'External debt stocks, total (DOD, current US$)', 'unit': '', 'source': {'id': '6', 'value': 'International Debt Statistics'}, 'sourceNote': 'Total external debt is debt owed to nonresidents repayable in currency, goods, or services. Total external debt is the sum of public, publicly guaranteed, and private nonguaranteed long-term debt, use of IMF credit, and short-term debt. Short-term debt includes all debt having an original maturity of one year or less and interest in arrears on long-term debt. Data are in current U.S. dollars.', 'sourceOrganization': 'World Bank, International Debt Statistics.', 'topics': [{'id': '3', 'value': 'Economy & Growth'}, {'id': '20', 'value': 'External Debt'}]}


In [22]:
# Print all selected countries
wbdata.get_country(incomelevel='LMY')

id    name
----  ------------------------------
AFG   Afghanistan
AGO   Angola
ALB   Albania
ARG   Argentina
ARM   Armenia
ASM   American Samoa
AZE   Azerbaijan
BDI   Burundi
BEN   Benin
BFA   Burkina Faso
BGD   Bangladesh
BGR   Bulgaria
BIH   Bosnia and Herzegovina
BLR   Belarus
BLZ   Belize
BOL   Bolivia
BRA   Brazil
BTN   Bhutan
BWA   Botswana
CAF   Central African Republic
CHN   China
CIV   Cote d'Ivoire
CMR   Cameroon
COD   Congo, Dem. Rep.
COG   Congo, Rep.
COL   Colombia
COM   Comoros
CPV   Cabo Verde
CRI   Costa Rica
CUB   Cuba
DJI   Djibouti
DMA   Dominica
DOM   Dominican Republic
DZA   Algeria
ECU   Ecuador
EGY   Egypt, Arab Rep.
ERI   Eritrea
ETH   Ethiopia
FJI   Fiji
FSM   Micronesia, Fed. Sts.
GAB   Gabon
GEO   Georgia
GHA   Ghana
GIN   Guinea
GMB   Gambia, The
GNB   Guinea-Bissau
GNQ   Equatorial Guinea
GRD   Grenada
GTM   Guatemala
GUY   Guyana
HND   Honduras
HTI   Haiti
IDN   Indonesia
IND   India
IRN   Iran, Islamic Rep.
IRQ   Iraq
JAM   Jamaica
JOR   Jordan
KAZ   Kaza

## Create dataframe
### Obs
- Here, I encountered the following error when specifying a source in the arguments, (source=6):
"RuntimeError: Got error 175 (Invalid format): The indicator was not found. It may have been deleted or archived."
- Time range is not required but I thought it maybe better not to leave rely on defaults

In [132]:
countries = [i['id'] for i in wbdata.get_country(incomelevel='LMY')]                                                                                            
indicators = {"DT.DOD.DECT.CD":"TotalExternalDebtUSD"}
timeSelection = (datetime.datetime(2009, 1, 1), datetime.datetime(2018, 12, 31))
df = wbdata.get_dataframe(indicators, data_date=timeSelection,country=countries, convert_date=True, keep_levels=False)   
#df = df.reset_index()
df.head(13)

TotalExternalDebtUSD
country     date                            
Afghanistan 2018-01-01          2.678760e+09
            2017-01-01          2.751987e+09
            2016-01-01          2.596050e+09
            2015-01-01          2.596917e+09
            2014-01-01          2.529865e+09
            2013-01-01          2.587775e+09
            2012-01-01          2.580632e+09
            2011-01-01          2.485331e+09
            2010-01-01          2.435845e+09
            2009-01-01          2.480214e+09
Angola      2018-01-01          6.321753e+10
            2017-01-01          5.917590e+10
            2016-01-01          5.782734e+10

For each country name
- slice the multi-index dataframe to get the country data
- rename col to country name
- get country data as pd.Series

In [155]:
df_list = []
for country_name in df.index.unique(level='country'):
    row = df.xs(country_name,level=0, axis=0)
    row = row.rename(columns = {'TotalExternalDebtUSD':country_name})
    df_list.append(row[country_name])
final_df = pd.concat(df_list, axis=1).transpose()
# This is me trying to rename the axis name but it doesnt work
#final_df = pd.concat(df_list, axis=1).transpose()
#final_df=final_df.rename_axis("country")
#final_df.reset_index
#final_df.set_index('country')
#final_df.index = final_df.index.rename("country")
#final_df.index.names = ["country"]
final_df

date,2018-01-01,2017-01-01,2016-01-01,2015-01-01,2014-01-01,2013-01-01,2012-01-01,2011-01-01,2010-01-01,2009-01-01
Afghanistan,2.678760e+09,2.751987e+09,2.596050e+09,2.596917e+09,2.529865e+09,2.587775e+09,2.580632e+09,2.485331e+09,2.435845e+09,2.480214e+09
Angola,6.321753e+10,5.917590e+10,5.782734e+10,4.925414e+10,4.645605e+10,4.407282e+10,3.685629e+10,3.437052e+10,2.679557e+10,2.027728e+10
Albania,9.875850e+09,9.801455e+09,8.516223e+09,8.447047e+09,8.512452e+09,8.647044e+09,7.384501e+09,6.484194e+09,5.436573e+09,4.605164e+09
Argentina,2.778273e+11,2.259253e+11,1.816385e+11,1.771846e+11,1.537938e+11,1.502250e+11,1.398777e+11,1.428846e+11,1.266424e+11,1.336951e+11
Armenia,1.072573e+10,1.022830e+10,9.855623e+09,8.831021e+09,8.555656e+09,8.681091e+09,7.640567e+09,7.410793e+09,6.307171e+09,4.934959e+09
...,...,...,...,...,...,...,...,...,...,...
Kosovo,2.325736e+09,2.504231e+09,2.124776e+09,2.158469e+09,2.242255e+09,2.220795e+09,2.002396e+09,1.699829e+09,1.455550e+09,1.725716e+09
"Yemen, Rep.",7.036510e+09,7.193100e+09,7.062872e+09,7.298887e+09,7.723142e+09,7.688886e+09,7.586177e+09,6.429107e+09,6.504445e+09,6.799196e+09
South Africa,1.740940e+11,1.749208e+11,1.439670e+11,1.247369e+11,1.424305e+11,1.415457e+11,1.478878e+11,1.190680e+11,1.084289e+11,8.001827e+10
Zambia,2.352627e+10,2.295432e+10,1.522065e+10,1.177854e+10,9.191820e+09,6.292316e+09,5.721667e+09,4.968062e+09,4.252911e+09,3.646364e+09
